## Funciones_Prepara_Prediccion.ipynb

### Objetivo

Declaración y desarrollo de funciones utilizadas en el proceso de predicción por estación para los días futuros.

### Descripción General de notebook

    Funciones:
    1. _baseMeteorologia
    2. _baseFestivos
    3. _prediccionMeteorologia
    4. _esFestivo
    5. _estacionEstado
    6. _estacionDataFechaNueva
    7. _dataBaseOriginal
    8. _dataBaseOriginalNula
    9. date_range
    10. _lecturaDatosOrigen
    11. _PredicionesRanking
    12. _EjecutaPrediccion

In [3]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import json, requests
import datetime
from datetime import timedelta
import io
import time
import joblib

import sys
sys.path.insert(0, '../1. Librerias Mongo')

from MongoDB_Connections import _connect_mongo
from MongoDB_Funciones_Consultas import _data_anio_mes
from MongoDB_Funciones_Consultas import _data_anio
from MongoDB_Funciones_Consultas import _data_anio_mes_por_estacion
from MongoDB_Funciones_Consultas import _data_anio_por_estacion
from MongoDB_Funciones_Consultas import _consulta_meteoUS_por_anio
from MongoDB_Funciones_Consultas import _consulta_meteoUS_por_anio_mes
from MongoDB_Funciones_Consultas import _consulta_meteoUS_por_anio_mes_dia
from MongoDB_Funciones_Consultas import _consulta_stations
from MongoDB_Funciones_Consultas import _consulta_stations_EstacionesMeteo
from MongoDB_Funciones_Consultas import _consulta_Demografia
from MongoDB_Funciones_Consultas import _consulta_EsFestivo

# %run "../1. Librerias Mongo/MongoDB_Funciones_Consultas.ipynb"

### 1. _baseMeteorologia

Consulta y formateo de BD de meteorología existente en colección en MongoDB Atlas

In [33]:
def _baseMeteorologia(db_conn):   

    data_Meteo = _consulta_meteoUS_full(db_conn)

    # Se agrega campo FECHA para join
    data_Meteo['FECHA'] =pd.to_datetime({'year': data_Meteo['ANIO'],
                                              'month': data_Meteo["MES"],
                                              'day':  data_Meteo['DIA']},
                                              format='%d-%m-%Y', errors='coerce')

    data_Meteo['TEMPERATURA'] = pd.to_numeric(data_Meteo['TEMPERATURA'])
    data_Meteo['VIENTO'] = pd.to_numeric(data_Meteo['VIENTO'])
    data_Meteo['PRESION'] = pd.to_numeric(data_Meteo['PRESION'])
    data_Meteo['HUMEDAD'] = pd.to_numeric(data_Meteo['HUMEDAD'])
    data_Meteo['PRECIPITACION_1h'] = pd.to_numeric(data_Meteo['PRECIPITACION_1h'])
    data_Meteo['PRECIPITACION_3h'] = pd.to_numeric(data_Meteo['PRECIPITACION_3h'])

    data_Meteo = data_Meteo.groupby(['ANIO', 'MES', 'DIA', 'FECHA']).agg(TEMP_MAX= ('TEMPERATURA','max'),
                                     TEMP_MIN = ('TEMPERATURA','min'),
                                     HUMEDAD = ('HUMEDAD','mean'),
                                     VIENTO = ('VIENTO','mean'),
                                     PRESION = ('PRESION', 'mean'),
                                     PRECIPITACION_1h = ('PRECIPITACION_1h', 'sum'),
                                     PRECIPITACION_3h = ('PRECIPITACION_3h', 'sum'),
                                     DESC_TIEMPO = ('DESC_TIEMPO', lambda x: x.value_counts().index[0]))

    data_Meteo.reset_index(inplace=True)
    
    return data_Meteo

### 2. _baseFestivos

Consulta y formateo de BD de días festivos existente en colección en MongoDB Atlas

In [ ]:
# FECHAS DE FESTIVOS
def _baseFestivos():
    df_festivos = pd.read_csv('../../data/Festivos_Madrid.csv', sep=';')
    # Se agrega campo FECHA para join
    df_festivos['FECHA'] =pd.to_datetime({'year': df_festivos['ANIO'],
                                              'month': df_festivos["MES"]
                                              ,'day':  df_festivos['DIA']},
                                              format='%d-%m-%Y', errors='coerce')
    return df_festivos

### 3. _prediccionMeteorologia

Retorno de datos meteorológicos según son consultados vía API a los servicios web de OpenWeather

In [34]:
# Recibe:
# fecha: fecha a consultar en formato datetime.datetime
# db_conn: conexión a Mongo, colección Meteo_US_NivelHora
def _prediccionMeteorologia(fecha, db_conn):
    
    # Los datos meteos se buscarán en OpenWeather si la fecha actual está en los siguientes 14 dias a partir de la fecha actual
    # Si la fecha es anterior a la actual los datos se buscarán en Mongo
    
    fechaIniQuincena = datetime.datetime.today().date() # Fecha de hoy
    fechaTerQuincena = fechaIniQuincena + datetime.timedelta(days=14) # 14 dias a partir de la fecha de hoy
    fecha_str = fecha.strftime('%d/%m/%Y')
    
    if ((fecha>=fechaIniQuincena) & (fecha<=fechaTerQuincena)):    # OpenWeather
    
        lat='40.41831'                                     # Latitude Madrid Center
        lon='-3.70275'                                     # Longitute Madrid Center
        units='metric'                                     # Temp: °C, Dist: meters
        lang='en'                                          # Weather description in english
        appid = APPID                                      # Secret token
        cnt = 16                                           # Cantidad de días que desea predecir
        req=f'https://api.openweathermap.org/data/2.5/forecast/daily?lat={lat}&lon={lon}&units={units}&lang={lang}&appid={appid}&cnt={cnt}'
        response = requests.get(req)
        # Convertir los datos json recibidos en un DataFrame pandas

        text = response.text
        weather= json.loads(text)
        weather_df_temp = pd.json_normalize(weather, record_path =['list'])
        
        # "weather" es un json anidado
        weather_df_temp2 = pd.json_normalize(weather['list'], record_path =['weather'])
        weather_df = pd.DataFrame()
        # Seleccionar y formatar los campos necesarios para el modelo
        weather_df['dt']= weather_df_temp['dt'].apply(datetime.datetime.fromtimestamp)
        weather_df['FECHA'] = weather_df['dt'].dt.strftime('%d/%m/%Y')
        weather_df[['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO', 'PRESION']] = \
            weather_df_temp[['temp.max','temp.min','humidity','speed','pressure']]
        
        # Cuando no tiene predicción de lluvia, la columna PRECIPITACION no existe, 
        # por eso las creamos con valor 0

#         if 'main.rain.1h' in weather_df_temp.columns:
#             weather_df['PRECIPITACION_1h'] = weather_df_temp['rain']
#         else:
#             weather_df['PRECIPITACION_1h'] = 0
        
        weather_df['PRECIPITACION_1h'] = weather_df_temp['rain'].fillna(0)
        weather_df['PRECIPITACION_3h']= weather_df['PRECIPITACION_1h']  
        weather_df['DESC_TIEMPO'] = weather_df_temp2['main']
        
        weather_out = weather_df.loc[(weather_df['FECHA'] == fecha_str)].drop(['dt','FECHA'], axis=1)
        
    else: # data en Mongo
        
        df_meteo = _consulta_meteoUS_por_anio_mes_dia(db_conn, fecha.year, fecha.month, fecha.day)
        
        weather_out = df_meteo.groupby(['ANIO','MES','DIA']).agg(
            TEMP_MAX= ('TEMPERATURA','max'),
            TEMP_MIN = ('TEMPERATURA','min'),
            HUMEDAD = ('HUMEDAD','mean'),
            VIENTO = ('VIENTO','mean'),
            PRESION = ('PRESION', 'mean'),
            PRECIPITACION_1h = ('PRECIPITACION_1h', 'sum'),
            PRECIPITACION_3h = ('PRECIPITACION_3h', 'sum'),
            DESC_TIEMPO = ('DESC_TIEMPO', lambda x: x.value_counts().index[0]))
        
        weather_out['HUMEDAD'] = round(weather_out['HUMEDAD']).astype(int)
        weather_out['VIENTO'] = round(weather_out['VIENTO']).astype(int)
        weather_out['PRESION'] = round(weather_out['PRESION']).astype(int)
        
        weather_out = weather_out.reset_index()
        weather_out = weather_out.drop(columns=['ANIO','MES','DIA'])        
        
    return weather_out


### 4. _esFestivo

Retorno 0 si el día consultado NO es festivo y 1 si es fectivo. El origen de la respuesta es una API de la Comunidad de Madrid.

In [ ]:
def _esFestivo(fecha):
    # Conectar con el Portal de datos abiertos del Ayuntamiento de Madrid para obtener los festivos

    url= 'https://datos.madrid.es/egob/catalogo/300082-0-calendario_laboral.csv'

    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=";")

    festivos = c[['Día','laborable / festivo / domingo festivo']].dropna(how='all')
    festivos['FECHA'] = pd.to_datetime(festivos['Día'], errors='coerce',format="%d/%m/%Y").dt.strftime('%d/%m/%Y')
    festivos = festivos.loc[(festivos["FECHA"] == fecha)]
    return 1 if festivos['laborable / festivo / domingo festivo'].item()=="Festivo" else 0

### 5. _estacionEstado

Retorno 1 si la estación BiciMad se encuentra activa al momento de la consulta, 0 si no se encuentra operativa, caso en el cual su predicción futura será 0.
El estado de la estación se consulta sobre la API de la AEMT del Ayuntamiento de Madrid.

In [ ]:
def _estacionEstado(estacion):
    # Credenciales de login usuario
    # Obtenidos con alta de aplicación en la plataforma MobilityLabs Madrid que pertenence a EMT

    XClientId = XCLIENTID
    passKey = PASSKEY
    # Obtener accessToken

    response = requests.get('https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/', headers={'X-ClientId': XClientId, 'passKey': passKey})
    text = response.text
    data = json.loads(text)
    accessToken = data['data'][0]['accessToken']
    
    # Hago la consulta en la API con el accessToken obtenido en el paso anterior

    response2 = requests.get('https://openapi.emtmadrid.es/v1/transport/bicimad/stations/', headers={'accessToken': accessToken})
    text = response2.text
    data2 = json.loads(text)['data']
    df_situation = pd.DataFrame(data2)
        
    if estacion not in df_situation['id'].values: # Si la estacion no existe en salida -> la estacion no existe en Bicimad
        estado = 0    
    else:
        estado = 1 if df_situation.loc[(df_situation["id"] == estacion)]['no_available'].item()==0 else 0
#         df_situation.loc[(df_situation["id"] == estacion)]['activate'].item()        
    
    return estado

### 6. _estacionDataFechaNueva

Construcción de DataFrame con FechaNueva que contiene los parámetros necesarios para ser entregados a la función final de predicción. Entre los parámetros necesarios:
    - Datos meteorológicos predichos en OpenWeather
    - EsFestivo
    - EsFinSemana
    otros

In [ ]:
def _estacionDataFechaNueva(idEstacion, fecha, df_Dummies, lst_CatCol, lst_NumCol, dbConnMeteo):
    #idEstacion, fecha, data_Meteo_in, df_festivos_in, df_Dummies, lst_CatCol, lst_NumCol):
    
    # Variable dia semana
    if fecha.isoweekday()==7:
        dia_semana=1
    else:
        dia_semana=fecha.isoweekday()+1

    # Variable Es_FinSemana
    if ((dia_semana==1) or (dia_semana==7)):
        es_finsemana = 1
    else:
        es_finsemana = 0

    # Variable TEMPORADA
    if fecha.month<=3:
        temporada='INVIERNO'
    elif fecha.month<=6:
        temporada='PRIMAVERA'
    elif fecha.month<=9:
        temporada='VERANO'
    else:
        temporada='OTONO'
    
#     meteo = data_Meteo_in[data_Meteo_in['FECHA']==fecha.strftime('%m/%d/%Y')]
#     es_festivo = df_festivos_in[df_festivos_in['FECHA']==fecha.strftime('%m/%d/%Y')]['FESTIVO'].iloc[0]

    meteo = _prediccionMeteorologia(fecha, dbConnMeteo)
    es_festivo = _esFestivo(fecha.strftime('%d/%m/%Y'))
    
    new_row_dict = {
        'ESTACION': idEstacion,
        'MES': fecha.month,
        'TEMPORADA': temporada,
        'DIA_SEMANA': dia_semana,
        'Es_Festivo': es_festivo,
        'Es_FinSemana': es_finsemana,
        'TEMP_MAX': float(meteo.TEMP_MAX.iloc[0]),
        'TEMP_MIN': float(meteo.TEMP_MIN.iloc[0]),
        'HUMEDAD': float(meteo.HUMEDAD.iloc[0]),
        'VIENTO': float(meteo.VIENTO.iloc[0]),
        'PRESION': float(meteo.PRESION.iloc[0]),
        'PRECIPITACION_1h': float(meteo.PRECIPITACION_1h.iloc[0]),
        'PRECIPITACION_3h': float(meteo.PRECIPITACION_3h.iloc[0]),
        'DESC_TIEMPO': meteo.DESC_TIEMPO.iloc[0]
    }

    fecha = pd.DataFrame(new_row_dict, index=[0])
    
    fecha['MES_sen'] = np.sin(2 * np.pi * fecha['MES'] / 11)
    fecha['MES_cos'] = np.cos(2 * np.pi * fecha['MES'] / 11)
    
    fecha['DIA_SEMANA'] = fecha['DIA_SEMANA'].astype('category')
    fecha['TEMPORADA'] = fecha['TEMPORADA'].astype('category')
    fecha['Es_Festivo'] = fecha['Es_Festivo'].astype('category')
    fecha['Es_FinSemana'] = fecha['Es_FinSemana'].astype('category')
    fecha['DESC_TIEMPO'] = fecha['DESC_TIEMPO'].astype('category')


    # Se agrega DF con valores para dummies
    fecha = pd.concat([fecha, df_Dummies])
    
    # Se generan campos dummies    
    fecha = pd.get_dummies(fecha, columns=lst_CatCol, drop_first=True)

    # Standarización de variables numéricas específicas
    df_NumCols = fecha[['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION','PRECIPITACION_1h','PRECIPITACION_3h']]

    scaler = StandardScaler()
    fecha[['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION','PRECIPITACION_1h','PRECIPITACION_3h']] = scaler.fit_transform(df_NumCols)
    
#     fecha = fecha.drop(['MES'], axis=1)
    
    # Se eliminan registros usados para generar variables dummies
    fecha = fecha.iloc[0]
    
    fecha = fecha[['MES_sen','MES_cos','TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION',
                   'PRECIPITACION_1h','PRECIPITACION_3h','Es_Festivo_1','Es_FinSemana_1',
                   'TEMPORADA_OTONO','TEMPORADA_PRIMAVERA','TEMPORADA_VERANO', 
                   'DIA_SEMANA_2','DIA_SEMANA_3','DIA_SEMANA_4','DIA_SEMANA_5','DIA_SEMANA_6','DIA_SEMANA_7',
                   'DESC_TIEMPO_Clouds','DESC_TIEMPO_Drizzle','DESC_TIEMPO_Fog','DESC_TIEMPO_Mist',
                   'DESC_TIEMPO_Rain','DESC_TIEMPO_Snow','DESC_TIEMPO_Thunderstorm']]
        
    fecha = fecha.astype(float)
        
    return pd.DataFrame(fecha).T

### 7. _dataBaseOriginal

Creación de DataFrame estandarizado con datos históricos reales

In [ ]:
# Generar DF con datos originales

def _dataBaseOriginal(filename):
    dataBase = pd.read_csv(filename, parse_dates=['FECHA'])
    
    dataBiciMad = dataBase.groupby(['ESTACION','ANIO','MES','DIA','TEMPORADA','DIA_SEMANA', 'Es_Festivo', 'Es_FinSemana',
                          ]).agg(DEMANDA=('DEMANDA', 'sum'),
                                 TEMP_MAX= ('TEMPERATURA','max'),
                                 TEMP_MIN = ('TEMPERATURA','min'),
                                 HUMEDAD = ('HUMEDAD','mean'),
                                 VIENTO = ('VIENTO','mean'),
                                 PRESION = ('PRESION', 'mean'),
                                 PRECIPITACION_1h = ('PRECIPITACION_1h', 'sum'),
                                 PRECIPITACION_3h = ('PRECIPITACION_3h', 'sum'),
                                 DESC_TIEMPO = ('DESC_TIEMPO', lambda x: x.value_counts().index[0]))
    
    dataBiciMad_DF = dataBiciMad.reset_index() 
    
    dataBiciMad_DF['MES_sen'] = np.sin(2 * np.pi * dataBiciMad_DF['MES'] / 11)
    dataBiciMad_DF['MES_cos'] = np.cos(2 * np.pi * dataBiciMad_DF['MES'] / 11)
    
    dataBiciMad_DF['DIA_SEMANA'] = dataBiciMad_DF['DIA_SEMANA'].astype('category')
    dataBiciMad_DF['Es_Festivo'] = dataBiciMad_DF['Es_Festivo'].astype('category')
    dataBiciMad_DF['Es_FinSemana'] = dataBiciMad_DF['Es_FinSemana'].astype('category')
    dataBiciMad_DF['DESC_TIEMPO'] = dataBiciMad_DF['DESC_TIEMPO'].astype('category')
    
    dataBiciMad_DF['DEMANDA'] =  np.log1p(dataBiciMad_DF['DEMANDA'])
    
    cols = dataBiciMad_DF.columns
    num_cols = dataBiciMad_DF._get_numeric_data().columns
    cat_cols = list(set(cols) - set(num_cols))
    
    dataBiciMad_DF = pd.get_dummies(dataBiciMad_DF, columns=cat_cols, drop_first=True)

    df_NumCols = dataBiciMad_DF[['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION','PRECIPITACION_1h','PRECIPITACION_3h']]

    scaler = StandardScaler()
    dataBiciMad_DF[['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION','PRECIPITACION_1h','PRECIPITACION_3h']] = scaler.fit_transform(df_NumCols)
    
    dataBiciMad_DF = dataBiciMad_DF.drop(['ANIO','MES','DIA'], axis=1)
    
    dataBiciMad_DF = dataBiciMad_DF[['ESTACION','DEMANDA','MES_sen','MES_cos','TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION',
                   'PRECIPITACION_1h','PRECIPITACION_3h','Es_Festivo_1','Es_FinSemana_1',
                   'TEMPORADA_OTONO','TEMPORADA_PRIMAVERA','TEMPORADA_VERANO', 
                   'DIA_SEMANA_2','DIA_SEMANA_3','DIA_SEMANA_4','DIA_SEMANA_5','DIA_SEMANA_6','DIA_SEMANA_7',
                   'DESC_TIEMPO_Clouds','DESC_TIEMPO_Drizzle','DESC_TIEMPO_Fog','DESC_TIEMPO_Mist',
                   'DESC_TIEMPO_Rain','DESC_TIEMPO_Snow','DESC_TIEMPO_Thunderstorm']]
    
    return dataBiciMad_DF


### 8. _dataBaseOriginalNula

Creación de DataFrame estandarizado con demanda nula pero con valores de variables explicativas existentes en base de datos históricos reales. 

In [ ]:
# Generar DF con datos originales pero ESTACION=0 y sin DEMANDA

def _dataBaseOriginalNula(filename):
    dataBase = pd.read_csv(filename, parse_dates=['FECHA'])
    
    dataBiciMad = dataBase.groupby(['ESTACION','ANIO','MES','DIA','TEMPORADA','DIA_SEMANA', 'Es_Festivo', 'Es_FinSemana',
                          ]).agg(
                                 TEMP_MAX= ('TEMPERATURA','max'),
                                 TEMP_MIN = ('TEMPERATURA','min'),
                                 HUMEDAD = ('HUMEDAD','mean'),
                                 VIENTO = ('VIENTO','mean'),
                                 PRESION = ('PRESION', 'mean'),
                                 PRECIPITACION_1h = ('PRECIPITACION_1h', 'sum'),
                                 PRECIPITACION_3h = ('PRECIPITACION_3h', 'sum'),
                                 DESC_TIEMPO = ('DESC_TIEMPO', lambda x: x.value_counts().index[0]))
    dataBiciMad = dataBiciMad.reset_index()    
    dataBiciMad['ESTACION'] = 0
    
    return dataBiciMad

### 9. date_range

Rango de dias entre 2 fechas específicas

In [1]:
def date_range(start, end):
    delta = end - start  # as timedelta
    days = [start + timedelta(days=i) for i in range(delta.days + 1)]
    return days

### 10. _lecturaDatosOrigen

Lectura de archivo csv

In [ ]:
def _lecturaDatosOrigen(filename):
    datosOrigen = pd.read_csv(filename)
    return datosOrigen

### 11. _PredicionesRanking

Creación de ranking para dias futuros de cluster basados en predicción de las estaciones que los componen. Se generan 4 rankings:
    - Ranking según lista de 5 clusters
    - Ranking según lista de 7 clusters
    - Ranking según lista de Barrios de Madrid
    - Ranking según lista de Distritos de Madrid

In [ ]:
# Objetivo: generar DF final de predicciones con ranking de cluster, barrios y distritos calculados en base a predicción predicha
def _PredicionesRanking(prediccionesIn, datosDF):
    
    # DESCARGA DE CLUSTER's y BARRIO/DISTRITO DE CADA ESTACION
    estaciones_cluster = datosDF[['ESTACION', 'CLUSTER_soloGeo5', 'CLUSTER_soloGeo7']].drop_duplicates()
    
    db_Estacion = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Estaciones')
    estaciones_barrios = _consulta_stations(db_Estacion)[['Id_Estacion', 'Barrio', 'Barrio_Nombre', 'Distrito', \
                                                      'Distrito_Nombre']].rename(columns={'Id_Estacion':'ESTACION'})
    
    estaciones = pd.merge(estaciones_cluster, estaciones_barrios)
    
    Predicciones = pd.merge(prediccionesIn, estaciones, how='left', left_on='ESTACION', right_on='ESTACION')
    
    
    # Creación de DF por tipo de Cluster: 5 Cluster, 7 Cluster y Barrio

    PrediccionCluster5 = Predicciones.groupby(['DIA','CLUSTER_soloGeo5']).agg(PREDICCION=('PREDICCION','sum'), NESTACIONES=('ESTACION', 'count'))
    PrediccionCluster5.reset_index(inplace=True)

    PrediccionCluster7 = Predicciones.groupby(['DIA','CLUSTER_soloGeo7']).agg(PREDICCION=('PREDICCION','sum'), NESTACIONES=('ESTACION', 'count'))
    PrediccionCluster7.reset_index(inplace=True)

    PrediccionClusterBarrio = Predicciones.groupby(['DIA','Barrio','Barrio_Nombre']).agg(PREDICCION=('PREDICCION','sum'), NESTACIONES=('ESTACION', 'count'))
    PrediccionClusterBarrio.reset_index(inplace=True)

    PrediccionClusterDistrito = Predicciones.groupby(['DIA','Distrito','Distrito_Nombre']).agg(PREDICCION=('PREDICCION','sum'), NESTACIONES=('ESTACION', 'count'))
    PrediccionClusterDistrito.reset_index(inplace=True)
    
    # Demanda total diaria
    PrediccionTotal = Predicciones.groupby('DIA').agg(PREDICCION_TOTAL=('PREDICCION','sum'))
    PrediccionTotal = PrediccionTotal.reset_index()
    
    
    # Por cada DF de predicción por tipo de cluster se agrega la columna de RATIO_FINAL = (PREDICCION/PREDICCION_TOTAL)/NESTACIONES

    PrediccionRatio5 = pd.merge(PrediccionCluster5, PrediccionTotal, how='left', left_on='DIA', right_on='DIA')
    PrediccionRatio5['RATIO'] = PrediccionRatio5['PREDICCION'] / PrediccionRatio5['PREDICCION_TOTAL']
    PrediccionRatio5['RATIO_FINAL'] = PrediccionRatio5['RATIO'] / PrediccionRatio5['NESTACIONES']

    PrediccionRatio7 = pd.merge(PrediccionCluster7, PrediccionTotal, how='left', left_on='DIA', right_on='DIA')
    PrediccionRatio7['RATIO'] = PrediccionRatio7['PREDICCION'] / PrediccionRatio7['PREDICCION_TOTAL']
    PrediccionRatio7['RATIO_FINAL'] = PrediccionRatio7['RATIO'] / PrediccionRatio7['NESTACIONES']

    PrediccionRatioBarrio = pd.merge(PrediccionClusterBarrio, PrediccionTotal, how='left', left_on='DIA', right_on='DIA')
    PrediccionRatioBarrio['RATIO'] = PrediccionRatioBarrio['PREDICCION'] / PrediccionRatioBarrio['PREDICCION_TOTAL']
    PrediccionRatioBarrio['RATIO_FINAL'] = PrediccionRatioBarrio['RATIO'] / PrediccionRatioBarrio['NESTACIONES']

    PrediccionRatioDistrito = pd.merge(PrediccionClusterDistrito, PrediccionTotal, how='left', left_on='DIA', right_on='DIA')
    PrediccionRatioDistrito['RATIO'] = PrediccionRatioDistrito['PREDICCION'] / PrediccionRatioDistrito['PREDICCION_TOTAL']
    PrediccionRatioDistrito['RATIO_FINAL'] = PrediccionRatioDistrito['RATIO'] / PrediccionRatioDistrito['NESTACIONES']
    
    # Se agrega columna rank en cada DF de tipo de cluster por DIA

    PrediccionRatio5['rank'] = PrediccionRatio5.groupby('DIA')['RATIO_FINAL'].rank('dense', ascending=False)
    PrediccionRatio7['rank'] = PrediccionRatio7.groupby('DIA')['RATIO_FINAL'].rank('dense', ascending=False)
    PrediccionRatioBarrio['rank'] = PrediccionRatioBarrio.groupby('DIA')['RATIO_FINAL'].rank('dense', ascending=False)
    PrediccionRatioDistrito['rank'] = PrediccionRatioDistrito.groupby('DIA')['RATIO_FINAL'].rank('dense', ascending=False)

    
    # Creación de DF Final que contiene Predicciones por Estación, cluster al cual pertenece y el ranking según tipo de cluster

    Predicciones = Predicciones.rename(columns={'PREDICCION':'PREDICCION_ESTACION'})

    PrediccionesFinal = pd.merge(Predicciones, 
                                 PrediccionRatio5[[
                                     'DIA','CLUSTER_soloGeo5','rank','PREDICCION','NESTACIONES',
                                     'PREDICCION_TOTAL','RATIO','RATIO_FINAL']], how='left',
                                   left_on=['DIA','CLUSTER_soloGeo5'], right_on=['DIA','CLUSTER_soloGeo5'])
    PrediccionesFinal.rename(columns={
        'rank':'RANK_CLUSTER5','PREDICCION':'PREDICCION_CLUSTER5','NESTACIONES':'NESTACIONES_CLUSTER5',
        'PREDICCION_TOTAL':'PREDICCION_TOTAL_CLUSTER5','RATIO':'RATIO_CLUSTER5','RATIO_FINAL':'RATIO_FINAL_CLUSTER5'}, inplace=True)



    PrediccionesFinal = pd.merge(PrediccionesFinal, 
                                 PrediccionRatio7[[
                                     'DIA','CLUSTER_soloGeo7','rank','PREDICCION','NESTACIONES',
                                     'PREDICCION_TOTAL','RATIO','RATIO_FINAL']], how='left',
                                   left_on=['DIA','CLUSTER_soloGeo7'], right_on=['DIA','CLUSTER_soloGeo7'])
    PrediccionesFinal.rename(columns={
        'rank':'RANK_CLUSTER7','PREDICCION':'PREDICCION_CLUSTER7','NESTACIONES':'NESTACIONES_CLUSTER7',
        'PREDICCION_TOTAL':'PREDICCION_TOTAL_CLUSTER7','RATIO':'RATIO_CLUSTER7','RATIO_FINAL':'RATIO_FINAL_CLUSTER7'}, inplace=True)



    PrediccionesFinal = pd.merge(PrediccionesFinal, PrediccionRatioBarrio[[
                                     'DIA','Barrio','rank','PREDICCION','NESTACIONES',
                                     'PREDICCION_TOTAL','RATIO','RATIO_FINAL']], how='left',
                                   left_on=['DIA','Barrio'], right_on=['DIA','Barrio'])
    PrediccionesFinal.rename(columns={
        'rank':'RANK_CLUSTER_BARRIO','PREDICCION':'PREDICCION_CLUSTER_BARRIO','NESTACIONES':'NESTACIONES_CLUSTER_BARRIO',
        'PREDICCION_TOTAL':'PREDICCION_TOTAL_CLUSTER_BARRIO','RATIO':'RATIO_CLUSTER_BARRIO',
        'RATIO_FINAL':'RATIO_FINAL_CLUSTER_BARRIO'}, inplace=True)

    PrediccionesFinal = pd.merge(PrediccionesFinal, PrediccionRatioDistrito[[
                                     'DIA','Distrito','rank','PREDICCION','NESTACIONES',
                                     'PREDICCION_TOTAL','RATIO','RATIO_FINAL']], how='left',
                                   left_on=['DIA','Distrito'], right_on=['DIA','Distrito'])
    PrediccionesFinal.rename(columns={
        'rank':'RANK_CLUSTER_DISTRITO','PREDICCION':'PREDICCION_CLUSTER_DISTRITO','NESTACIONES':'NESTACIONES_CLUSTER_DISTRITO',
        'PREDICCION_TOTAL':'PREDICCION_TOTAL_CLUSTER_DISTRITO','RATIO':'RATIO_CLUSTER_DISTRITO',
        'RATIO_FINAL':'RATIO_FINAL_CLUSTER_DISTRITO'}, inplace=True)
    
    return PrediccionesFinal

### 12. _EjecutaPrediccion

Procedimiento para el cálculo de predicción diaria para cada una de las estaciones BiciMad del proyecto a partir de "fechaStart" y para los siguientes "diasDelta".
El resultado es un DataFrame con el siguiente formato: [ESTACION, ANIO, MES, DIA, PREDICCION]

In [ ]:
# Parámetros:
#   - fechaStart: fecha inicio de predicción en formato string dd/mm/yyyy
#   - DiasDelta: cantidad de días a predecir posterior a la fechaStart

def _EjecutaPrediccion(fechaStart, diasDelta, datosOrigen):    
    
    # CONEXION A COLLECTION DE TIEMPO EN MONGODB
    db_Meteo = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 
                          'Meteo_US_NivelHora')
    
    # lISTA DE ESTACIONES
    df_estaciones = datosOrigen.ESTACION.unique()
    df_estaciones = df_estaciones[5:15]
    df_estaciones = np.sort(df_estaciones)
    
    estaciones_cluster = datosOrigen[['ESTACION', 'CLUSTER_soloGeo5', 'CLUSTER_soloGeo7']].drop_duplicates()
        
    
    # GENERACION DE DF CON DATOS NULOS y COLUMNAS CATEGORICAS Y NUMERICAS
    dataDummy = _dataBaseOriginalNula("../../Data/DataFrame_Final_Cierre_Cluster_2017_2019.csv")
    cat_cols= ['MES', 'DIA_SEMANA', 'TEMPORADA', 'Es_Festivo', 'Es_FinSemana','DESC_TIEMPO']
    num_cols= ['TEMP_MAX','TEMP_MIN','HUMEDAD','VIENTO','PRESION']
    
    
    # PREDICCION DE DEMANDA POR ESTACION y DIA
    fecha_inicio = datetime.datetime.strptime(fechaStart, '%d/%m/%Y').date()
    fecha_termino = fecha_inicio + datetime.timedelta(days=int(diasDelta)-1)

    Fechas = pd.DataFrame()
    Predicciones = pd.DataFrame()

    t_ini = time.time()

    for estacion in df_estaciones:

        iEstadoEstacion = _estacionEstado(estacion)

        print('ESTACION:'+str(estacion)+'; ESTADO:'+str(iEstadoEstacion))

        model = joblib.load('../Modelos/Modelo_'+ str(estacion) +'.pkl')

        rango_fechas = date_range(fecha_inicio, fecha_termino)
        for fecha in rango_fechas:
            print(fecha)

            FechaAPredecir = _estacionDataFechaNueva(estacion, fecha, dataDummy, cat_cols, num_cols, db_Meteo)

            if Fechas.empty:
                Fechas = FechaAPredecir.copy()
            else:
                Fechas = pd.concat([Fechas, FechaAPredecir])            

            # ESTACION ACTIVA
            if (iEstadoEstacion==1):

                # Prediccion según modelo entrenado para cada una de las estacion
                prediccion = model.predict(FechaAPredecir)[0]
                prediccion = pd.DataFrame([str(estacion), fecha.year, fecha.month, fecha.day, prediccion]).T
                predExp = round(np.expm1(prediccion[4].item()))  # Se aplica exponencial para eliminar el valor log utilizado para entrenar
                prediccion[4] = predExp

                # Se crea Predicciones sólo en primera ejecución de los loops
                if Predicciones.empty:                
                    Predicciones = prediccion.copy()
                else:
                    Predicciones = pd.concat([Predicciones, prediccion])


            # ESTACION INACTIVA
            else:
                pred_nula = {0:estacion,1:fecha.year,2:fecha.month,3:fecha.day,4:0}
                pred_nula = pd.DataFrame(data=pred_nula, index=[0])

                # Se crea Predicciones sólo en primera ejecución de los loops
                if Predicciones.empty:
                    print('ini_nula')
                    Predicciones = pred_nula.copy()
                else:
                    Predicciones = pd.concat([Predicciones, pred_nula])

    t_end = time.time()
    print ((t_end - t_ini)/60)

    Predicciones.rename(columns={0:'ESTACION', 1:'ANIO', 2:'MES', 3:'DIA', 4:'PREDICCION'}, inplace=True)
    Predicciones['ESTACION']=Predicciones['ESTACION'].astype(int)
    Predicciones['ANIO']=Predicciones['ANIO'].astype(int)
    Predicciones['MES']=Predicciones['MES'].astype(int)
    Predicciones['DIA']=Predicciones['DIA'].astype(int)
    Predicciones['PREDICCION']=Predicciones['PREDICCION']#.astype(int)
    
    return Predicciones